In [63]:
import pandas as pd
from datetime import datetime


def heuristic_sort(csv_file):

    df = pd.read_csv(csv_file)

    fill_factors = ['Lead Years In Company', 'Lead Months In Company', 'Lead Months In Position', 'Lead Years In Position']

    for factor in fill_factors:
        df[factor].fillna(0, inplace=True)

    # Calculate total months in position
    df['Lead Total Months In Company'] = df['Lead Years In Company'] * 12 + df['Lead Months In Company']
    df['Lead Total Months In Position'] = df['Lead Years In Position'] * 12 + df['Lead Months In Position']

    drop_columns = [
        'Lead Location Raw', 'Lead Location Country Name', 'Lead Location Country Code', 'Lead Location State', 'Lead Location City', 'Lead Location Timezone', 'Lead Location Timezone Offset', 'Lead Location Postal Code', 'Lead Premium', 'Lead Li Picture Url', 'Company Location Country Code', 'Company Location State', 'Company Location City', 'Company Location Timezone', 'Company Location Timezone Offset', 'Company Location Postal Code'
    ]
    df.drop(columns=drop_columns, inplace=True)
    
    current_year = datetime.now().year
    
    negative_descriptors = ['consulting', 'consultant', 'llc', 'venture capital']
    positive_descriptors = ['crypto', 'cryptocurrency', 'artificial intelligence', 'machine learning', 'pharmaceuticals',
                           'pharmaceutic', 'pharmaceutical', 'medical device', 'med device', 'biotech']

    def score_row(row):
        score = 0

        # Factor 1: Closeness of the year to the current year (trying to avoid companies that have been around for a long time)
        row['factor_1_score'] = -(current_year - row['Company Founded In'])

        # Factor 2: Company Li Description not null
        if pd.isnull(row['Company Li Description']):
            row['factor_2_score'] = 0
        else:
            descriptor_score = 0
            description = row['Company Li Description'].lower()
            if any(descriptor in description for descriptor in negative_descriptors):
                descriptor_score = -2
            elif any(descriptor in description for descriptor in positive_descriptors):
                descriptor_score = 1
            row['factor_2_score'] = descriptor_score
                    

        # Factor 3: Difference in Lead Total Months In Company and Lead Total Months In Position
        months_difference = abs(row['Lead Total Months In Company'] - row['Lead Total Months In Position'])
        if months_difference != 0:
            # Red flag if these two numbers are not the same
            row['factor_3_score'] = -1
        else:
            row['factor_3_score'] = 0

        # Factor 4: Closeness of Lead Total Months In Company and the time the company has been around
        factor_4_score = 1 - (abs(row['Lead Total Months In Company'] - (current_year - row['Company Founded In'])))
        row['factor_4_score'] = factor_4_score

        # Factor 5: Closeness of Lead Total Months In Company and 12 months
        factor_5_score = -(abs(row['Lead Total Months In Company'] - 12))
        row['factor_5_score'] = factor_5_score

         # Factor 6: Check if company name is LLC
        row['factor_6_score'] = -2 if 'LLC' in str(row['Company Name']) else 0
 
        # Factor 7: Consider the size of the company
        company_size = row['Company Size']
        if company_size in ['2-10', '11-50']:
            row['factor_7_score'] = 1
        elif pd.isnull(company_size) or company_size in ['51-200', '201-500']:
            row['factor_7_score'] = 0
        else:
            row['factor_7_score'] = -2
        
        # Factor 8: Remove companies in venture capital and business consulting
        row['factor_8_score'] = -2 if 'Business Consulting and Services' in str(row['Company Industry']) else -2 if 'Venture Capital' in str(row['Company Industry']) else 0

        # Factor 9: Remove llc LinkedIn companies
        if pd.notnull(row['Company Linkedin']):
            row['factor_9_score'] = -1 if 'llc' in row['Company Linkedin'].lower() else 0
        else:
            row['factor_9_score'] = 0
        return row

    # Apply scoring function to each row
    df = df.apply(score_row, axis=1)
    
    factors = [col for col in df.columns if col.startswith('factor') and col.endswith('score')]
    
    # Normalize the factors
    for factor in factors:
        min_val = df[factor].min()
        max_val = df[factor].max()
        df[factor] = (df[factor] - min_val) / (max_val - min_val)

    # Sum normalized factors to get the total score
    df['Score'] = df[factors].sum(axis=1)

    # Drop factor columns
    df.drop(columns=factors+['Lead Total Months In Company', 'Lead Total Months In Position'], inplace=True)

    # Sort DataFrame by score
    df_sorted = df.sort_values(by='Score', ascending=False)
    
    # Fill na
    df_sorted.fillna("NA", inplace=True)
    
    #Save to CSV
    today_date = datetime.today().strftime('%Y-%m-%d')

    df_sorted.to_csv('heuristic_sorted_' + today_date + '.csv', index=False)

    print("CSV file saved successfully.")

In [64]:
if __name__ == "__main__":

    csv_file = "Test-Extraction-1-26-2314853-all-leads.csv"
    heuristic_sort(csv_file)

CSV file saved successfully.
